In [2]:
import requests

def download_kaggle_dataset(url, output_file):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(output_file, 'wb') as file:
                file.write(response.content)
            print(f"Base de dados baixada com sucesso para {output_file}")
        else:
            print(f"Erro ao baixar a base de dados. Status code: {response.status_code}")
    except Exception as e:
        print(f"Erro durante a requisição: {e}")

# URL da base de dados de testes no Kaggle
kaggle_url = "https://www.kaggle.com/datasets/josephlovins/supermarket-transactions?select=Supermarket+Transactions.xlsx"

# Nome do arquivo de saída
output_filename = "supermarket_transaction.csv"

# Chama a função para baixar a base de dados
download_kaggle_dataset(kaggle_url, output_filename)


Base de dados baixada com sucesso para supermarket_transaction.csv
